In [27]:
import pandas as pd
from sqlalchemy import create_engine # database connection

In [28]:
# Initializes database with file movieRating.sqlite
disk_engine = create_engine('sqlite:///movieRating.sqlite') 

In [66]:
df = pd.read_sql_query('Select * from movie', disk_engine)
df.head(20)

,mID,title,year,director
0,101,Gone with the wind,1939,Victor Fleming
1,102,Star Wars,1977,George Lucas
2,103,The Sound of Music,1965,Robert Wise
3,104,E.T.,1982,Steven Spielberg
4,105,Titanic,1997,James Cameron
5,106,Snow White,1937,None
6,107,Avatar,2009,James Cameron
7,108,Raiders of the Lost Ark,1981,Steven Spielberg


In [67]:
df = pd.read_sql_query('Select * from rating', disk_engine)
df.head(20)

,rID,mID,stars,ratingDate
0,201,101,2,2011-01-22
1,201,101,4,2011-01-27
2,202,106,4,None
3,203,103,2,2011-01-20
4,203,108,4,2011-01-12
5,203,108,2,2011-01-30
6,204,101,3,2011-01-09
7,205,103,3,2011-01-27
8,205,104,2,2011-01-22
9,205,108,4,None


In [68]:
df = pd.read_sql_query('Select * from reviewer', disk_engine)
df.head(20)

,rID,name
0,201,Sarah Martinez
1,202,Daniel Lewis
2,203,Brittany Harris
3,204,Mike Anderson
4,205,Chris Jackson
5,206,Elizabeth Thomas
6,207,James Cameron
7,208,Ashley White


In [40]:
# Find all years that have a movie that received a rating of 4 or 5, and sort them in increasing order.
df=pd.read_sql_query('Select distinct Movie.year From Movie, Rating Where Movie.mID=Rating.mID and Rating.stars >= 4 order by movie.year', disk_engine)
df.head()

,year
0,1937
1,1939
2,1981
3,2009


In [39]:
# Find the titles of all movies directed by Steven Spielberg. 
df=pd.read_sql_query('SELECT title '
                     'FROM movie '
                     'WHERE movie.director = "Steven Spielberg" ', disk_engine)
df.head()

,title
0,E.T.
1,Raiders of the Lost Ark


In [47]:
# Find the titles of all movies that have no ratings. 
df=pd.read_sql_query('SELECT title '
                     'FROM movie '
                     'WHERE mID NOT IN (SELECT movie.mID FROM movie, rating WHERE movie.mID=rating.mID)', disk_engine)
df.head()

,title
0,Star Wars
1,Titanic


In [50]:
# Some reviewers didn't provide a date with their rating. 
# Find the names of all reviewers who have ratings with a NULL value for 
# the date. 
df=pd.read_sql_query('SELECT name '
                  'FROM reviewer '
                  'WHERE rID in (select rID from rating where ratingDate is null)', disk_engine) 
df.head()

,name
0,Daniel Lewis
1,Chris Jackson


In [60]:
# Write a query to return the ratings data in a more readable format: 
# reviewer name, movie title, stars, and ratingDate. 
#Also, sort the data, first by reviewer name, then by movie title, and lastly by number of stars. 
df=pd.read_sql_query('select distinct reviewer.name, movie.title, rating.stars, rating.ratingDate '
                     'from movie, reviewer, rating '
                     'where movie.mID=rating.mID and rating.rID=reviewer.rID '
                     'order by reviewer.name, movie.title, rating.stars', disk_engine)
df.head(20)

,name,title,stars,ratingDate
0,Ashley White,E.T.,3,2011-01-02
1,Brittany Harris,Raiders of the Lost Ark,2,2011-01-30
2,Brittany Harris,Raiders of the Lost Ark,4,2011-01-12
3,Brittany Harris,The Sound of Music,2,2011-01-20
4,Chris Jackson,E.T.,2,2011-01-22
5,Chris Jackson,Raiders of the Lost Ark,4,None
6,Chris Jackson,The Sound of Music,3,2011-01-27
7,Daniel Lewis,Snow White,4,None
8,Elizabeth Thomas,Avatar,3,2011-01-15
9,Elizabeth Thomas,Snow White,5,2011-01-19


In [61]:
# For all cases where the same reviewer rated the same movie twice and gave it a higher 
# rating the second time, return the reviewer's name and the title of the movie. 
df=pd.read_sql_query('select reviewer.name, movie.title '
                     'from reviewer, movie, (select R1.rID, R1.mID from rating R1, rating R2 where R1.rID=R2.rID and R1.mID=R2.mID and R1.stars<R2.stars and R1.ratingDate<R2.ratingDate) test '
                     'where test.rID=reviewer.rID and test.mID=movie.mID', disk_engine)
df.head(5)

,name,title
0,Sarah Martinez,Gone with the wind


In [62]:
# For each movie that has at least one rating, find the highest number 
# of stars that movie received. Return the movie title and number of stars. Sort by movie title. 
df=pd.read_sql_query('select movie.title, M.maxStr '
                     'from movie,(select rating.mID as rtMID, max(rating.stars) as maxStr from rating group by mID) M '
                     'Where movie.mID=M.rtMID '
                     'order by movie.title ', disk_engine)
df.head()

,title,maxStr
0,Avatar,5
1,E.T.,3
2,Gone with the wind,4
3,Raiders of the Lost Ark,4
4,Snow White,5


In [63]:
# For each movie, return the title and the 'rating spread', 
#that is, the difference between highest and lowest ratings given to 
#that movie. Sort by rating spread from highest to lowest, then by movie title.
df=pd.read_sql_query('select movie.title, max(rating.stars)-min(rating.stars) as sprd '
                     'from movie, rating '
                     'where movie.mID=rating.mID '
                     'group by movie.mID '
                     'order by sprd desc, movie.title', disk_engine)
df.head()

,title,sprd
0,Avatar,2
1,Gone with the wind,2
2,Raiders of the Lost Ark,2
3,E.T.,1
4,Snow White,1
